In [1]:
import numpy as np
import pandas as pd
import sqlite3

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
con = sqlite3.connect("test.sqlite")

In [3]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('preferences',), ('options',), ('picks',), ('cards',)]


## Implementaiton

In [30]:
from make_analytic_table import AnalyticTableConstructor

atc = AnalyticTableConstructor(db_path="test.sqlite")
X, y, y_names_mapping = atc.make_analytic_base_table()

In [27]:
X.head()

options_angel_of_the_dawn  \
draft_id                             drafter pick_number                              
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                    2   
                                             1                                    1   
                                             2                                    2   
                                             3                                    0   
                                             4                                    1   

                                                          options_luminous_bonds  \
draft_id                             drafter pick_number                           
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                 0   
                                             1                                 0   
                                             2                                 2   
                                             3                                 1   
                                             4                                 0   

                                                          options_pegasus_courser  \
draft_id                             drafter pick_number                            
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                  1   
                                             1                                  0   
                                             2                                  0   
                                             3                                  0   
                                             4                                  0   

                                                          options_hieromancer's_cage  \
draft_id                             drafter pick_number                               
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                     2   
                                             1                                     0   
                                             2                                     0   
                                             3                                     0   
                                             4                                     0   

                                                          options_leonin_warleader  \
draft_id                             drafter pick_number                             
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                   0   
                                             1                                   0   
                                             2                                   0   
                                             3                                   0   
                                             4                                   0   

                                                          options_aviation_pioneer  \
draft_id                             drafter pick_number                             
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                   0   
                                             1                                   0   
                                             2                                   0   
                                             3                                   0   
                                             4                                   1   

                                                          options_essence_scatter  \
draft_id                             drafter pick_number                            
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                  1   
                                             1                                  0   
                                             2                                  0   
               

In [28]:
y.head()

draft_id                              drafter  pick_number
d82a6ded-407e-4ba7-be27-945ec7b9233e  0        0              19
                                               1              18
                                               2              18
                                               3              15
                                               4              15
dtype: int64

In [29]:
y_names_mapping

{0: 'Angel of the Dawn',
 1: 'Luminous Bonds',
 2: 'Pegasus Courser',
 3: "Hieromancer's Cage",
 4: 'Leonin Warleader',
 5: 'Aviation Pioneer',
 6: 'Essence Scatter',
 7: 'Salvager of Secrets',
 8: 'Horizon Scholar',
 9: 'Djinn of Wishes',
 10: 'Doomed Dissenter',
 11: "Lich's Caress",
 12: 'Skymarch Bloodletter',
 13: 'Murder',
 14: 'Demon of Catastrophes',
 15: 'Electrify',
 16: 'Viashino Pyromancer',
 17: 'Sparktongue Dragon',
 18: 'Lightning Strike',
 19: 'Demanding Dragon',
 20: 'Druid of the Cowl',
 21: 'Giant Spider',
 22: 'Llanowar Elves',
 23: 'Dryad Greenseeker',
 24: 'Pelakka Wurm'}

## Exploratory Work

### Pick Options

In [4]:
options = pd.read_sql_query("select * from options;", con)
options = options.set_index(['draft_id', 'drafter', 'pick_number'])

In [5]:
options

Angel of the Dawn  \
draft_id                             drafter pick_number                      
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                            2   
                                             1                            1   
                                             2                            2   
                                             3                            0   
                                             4                            1   
                                             5                            0   
                                             6                            0   
                                             7                            0   
                                             8                            0   
                                             9                            0   
                                             10                           0   
                                             11                           0   
                                             12                           0   
                                             13                           0   
                                             14                           1   
                                             15                           1   
                                             16                           1   
                                             17                           0   
                                             18                           1   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           1   
                                             29                           1   
...                                                                     ...   
                                     7       12                           0   
                                             13                           0   
                                             14                           1   
                                             15                           1   
                                             16                           0   
                                             17                           1   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           1   
                                             29                           0   
                             

There should be a decreasing number of options as we progress through the draft, repeating in a cycle of length 14 (since there are 14 cards in each pack.

In [6]:
(options.sum(axis=1) == 14 - options.index.get_level_values('pick_number').values % 14).all()

True

To avoid duplicate column names, we will sanitize the names and add a prefix.

In [7]:
def sanitizer(prefix=''):
    def sanitize_name(name):
        return prefix + '_' + name.lower().replace(' ', '_')
    return sanitize_name

In [8]:
sanitizer(prefix='options')('Angel Of The Dawn')

'options_angel_of_the_dawn'

In [9]:
options.columns = options.columns.map(sanitizer(prefix='options'))

### Current Cards

In [10]:
cards = pd.read_sql_query("select * from cards;", con)
cards = cards.set_index(['draft_id', 'drafter', 'pick_number'])

In [11]:
cards

Angel of the Dawn  \
draft_id                             drafter pick_number                      
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                            0   
                                             1                            0   
                                             2                            0   
                                             3                            0   
                                             4                            0   
                                             5                            0   
                                             6                            0   
                                             7                            0   
                                             8                            0   
                                             9                            0   
                                             10                           0   
                                             11                           0   
                                             12                           0   
                                             13                           0   
                                             14                           0   
                                             15                           0   
                                             16                           0   
                                             17                           0   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           0   
                                             29                           0   
...                                                                     ...   
                                     7       12                           0   
                                             13                           0   
                                             14                           0   
                                             15                           0   
                                             16                           0   
                                             17                           0   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           0   
                                             29                           0   
                             

We should have an increasing number of cards as we progress through the draft. The number of cards held by each player should be equal to the pick number of the draft.

In [12]:
(cards.sum(axis=1) == cards.index.get_level_values('pick_number').values).all()

True

In [13]:
cards.columns = cards.columns.map(sanitizer(prefix='cards'))

### Draft Picks

Load the picks table, and convert it from a one-hot-encoded array into a series that tracks the pick by index (i.e., each card is assigned an integer).

In [14]:
picks = pd.read_sql_query("select * from picks;", con)
picks = picks.set_index(['draft_id', 'drafter', 'pick_number'])

In [22]:
dict(enumerate(picks.columns))

{0: 'Angel of the Dawn',
 1: 'Luminous Bonds',
 2: 'Pegasus Courser',
 3: "Hieromancer's Cage",
 4: 'Leonin Warleader',
 5: 'Aviation Pioneer',
 6: 'Essence Scatter',
 7: 'Salvager of Secrets',
 8: 'Horizon Scholar',
 9: 'Djinn of Wishes',
 10: 'Doomed Dissenter',
 11: "Lich's Caress",
 12: 'Skymarch Bloodletter',
 13: 'Murder',
 14: 'Demon of Catastrophes',
 15: 'Electrify',
 16: 'Viashino Pyromancer',
 17: 'Sparktongue Dragon',
 18: 'Lightning Strike',
 19: 'Demanding Dragon',
 20: 'Druid of the Cowl',
 21: 'Giant Spider',
 22: 'Llanowar Elves',
 23: 'Dryad Greenseeker',
 24: 'Pelakka Wurm'}

In [15]:
pick_idx = pd.Series(np.argmax(picks.values, axis=1), index=picks.index)

In [16]:
pick_idx

draft_id                              drafter  pick_number
d82a6ded-407e-4ba7-be27-945ec7b9233e  0        0              19
                                               1              18
                                               2              18
                                               3              15
                                               4              15
                                               5              15
                                               6              16
                                               7              18
                                               8              16
                                               9              16
                                               10             16
                                               11             17
                                               12             18
                                               13             21
                               

In [42]:
cards - picks

Angel of the Dawn  \
draft_id                             drafter pick_number                      
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                          NaN   
                                             1                          NaN   
                                             2                          NaN   
                                             3                          NaN   
                                             4                          NaN   
                                             5                          NaN   
                                             6                          NaN   
                                             7                          NaN   
                                             8                          NaN   
                                             9                          NaN   
                                             10                         NaN   
                                             11                         NaN   
                                             12                         NaN   
                                             13                         NaN   
                                             14                         NaN   
                                             15                         NaN   
                                             16                         NaN   
                                             17                         NaN   
                                             18                         NaN   
                                             19                         NaN   
                                             20                         NaN   
                                             21                         NaN   
                                             22                         NaN   
                                             23                         NaN   
                                             24                         NaN   
                                             25                         NaN   
                                             26                         NaN   
                                             27                         NaN   
                                             28                         NaN   
                                             29                         NaN   
...                                                                     ...   
                                     7       12                         NaN   
                                             13                         NaN   
                                             14                         NaN   
                                             15                         NaN   
                                             16                         NaN   
                                             17                         NaN   
                                             18                         NaN   
                                             19                         NaN   
                                             20                         NaN   
                                             21                         NaN   
                                             22                         NaN   
                                             23                         NaN   
                                             24                         NaN   
                                             25                         NaN   
                                             26                         NaN   
                                             27                         NaN   
                                             28                         NaN   
                                             29                         NaN   
                             

In [48]:
np.all((cards.iloc[1:, :].values - picks.iloc[:-1, :].values) == cards.iloc[:-1, :], axis=1)

draft_id                              drafter  pick_number
d82a6ded-407e-4ba7-be27-945ec7b9233e  0        0              True
                                               1              True
                                               2              True
                                               3              True
                                               4              True
                                               5              True
                                               6              True
                                               7              True
                                               8              True
                                               9              True
                                               10             True
                                               11             True
                                               12             True
                                               13             True
   

### Joining

In [17]:
joined = pd.merge(cards, options, left_index=True, right_index=True)

In [18]:
joined["pick"] = pick_idx

In [19]:
joined.head()

cards_angel_of_the_dawn  \
draft_id                             drafter pick_number                            
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                  0   
                                             1                                  0   
                                             2                                  0   
                                             3                                  0   
                                             4                                  0   

                                                          cards_luminous_bonds  \
draft_id                             drafter pick_number                         
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                               0   
                                             1                               0   
                                             2                               0   
                                             3                               0   
                                             4                               0   

                                                          cards_pegasus_courser  \
draft_id                             drafter pick_number                          
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                0   
                                             1                                0   
                                             2                                0   
                                             3                                0   
                                             4                                0   

                                                          cards_hieromancer's_cage  \
draft_id                             drafter pick_number                             
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                   0   
                                             1                                   0   
                                             2                                   0   
                                             3                                   0   
                                             4                                   0   

                                                          cards_leonin_warleader  \
draft_id                             drafter pick_number                           
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                 0   
                                             1                                 0   
                                             2                                 0   
                                             3                                 0   
                                             4                                 0   

                                                          cards_aviation_pioneer  \
draft_id                             drafter pick_number                           
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                 0   
                                             1                                 0   
                                             2                                 0   
                                             3                                 0   
                                             4                                 0   

                                                          cards_essence_scatter  \
draft_id                             drafter pick_number                          
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                0   
                                             1                                0   
                                             2                                0   
                                             3                                0   
                          